In [1]:
!pip install numpy pandas

In [140]:
import pandas as pd

crime = pd.read_csv(r"\Users\singh\Downloads\crime_with_weather.csv")

C:\Users\singh\AppData\Local\Temp\ipykernel_35856\3148661739.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  crime = pd.read_csv(r"\Users\singh\Downloads\crime_with_weather.csv")


In [141]:
crime.columns

Index(['object_id', 'primary_key', 'case_number', 'district', 'ucr_desc',
       'time_group', 'reported_date', 'offense_month', 'offense_day',
       'time_block', 'dow_name', 'dow', 'hour_of_day', 'days_ago',
       'offense_date', 'statute', 'city', 'zip', 'stat_desc', 'address_public',
       'std_parcelpin', 'ward', 'census_tract', 'census_tract_geoid',
       'census_block_group', 'census_bg_geoid', 'census_block',
       'census_block_geoid', 'lat', 'lon', 'offense_year', 'geoid', 'date',
       'temp_max', 'temp_min', 'daylight_duration', 'precipitation_sum',
       'precipitation_hours'],
      dtype='object')

In [142]:
cols = ['census_tract_geoid', 'census_block_group', 'census_bg_geoid', 'census_block_geoid', 'lat', 'lon', 'std_parcelpin', 'city',
        'zip', 'address_public', 'ward', 'object_id', 'primary_key', 'case_number', 'district', 'reported_date', 'dow_name', 'statute', 
        'stat_desc', 'time_group', 'date', 'days_ago', 'geoid']

crime = crime.drop(columns=cols)

In [143]:
cols = ['census_tract', 'census_block']

# Loop through each column and count 'Not Located' occurrences
for col in cols:
    count_not_located = (crime[col] == 'Not Located').sum()
    print(f"'{col}' has {count_not_located} 'Not Located' entries")

'census_tract' has 0 'Not Located' entries
'census_block' has 0 'Not Located' entries


In [144]:
# Function to create extracted month and day columns
def add_extracted_date_columns(crime):
    crime['offense_date'] = pd.to_datetime(crime['offense_date'], errors='coerce')
    crime.dropna(subset=['offense_date'], inplace=True)  # Drop rows where 'offense_date' is NaT
    crime['extracted_month'] = crime['offense_date'].dt.month
    crime['extracted_day'] = crime['offense_date'].dt.day
    return crime

# Function to check for discrepancies
def date_discrepancy(crime, month, day):
    inconsistent_records = crime[(crime[month] != crime['extracted_month']) | (crime[day] != crime['extracted_day'])]
    return inconsistent_records.shape[0]

# Add extracted month and day columns first
crime = add_extracted_date_columns(crime)

# Checking discrepancies before dropping 'offense_month' and 'offense_day'
initial_discrepancies = date_discrepancy(crime, 'offense_month', 'offense_day')
print(f"Initial discrepancies: {initial_discrepancies}")

# Dropping the original columns now
crime = crime.drop(columns=['offense_month', 'offense_day'])

# Perform a final discrepancy check to confirm consistency (should be zero)
final_discrepancies = date_discrepancy(crime, 'extracted_month', 'extracted_day')
print(f"Final discrepancies: {final_discrepancies}")

Initial discrepancies: 128136
Final discrepancies: 0


In [146]:
# Only use entries from years between 2018-2022 (Census data is for those years)
# Years before do not have enough values

crime = crime[(crime['offense_year'] > 2017) & (crime['offense_year'] < 2023)]
crime.shape[0]

377010

In [147]:
# Look at rows with null entries

rows_with_null = crime[crime.isnull().any(axis=1)]
rows_with_null

,ucr_desc,time_block,dow,hour_of_day,offense_date,census_tract,census_block,offense_year,temp_max,temp_min,daylight_duration,precipitation_sum,precipitation_hours,extracted_month,extracted_day


In [148]:
duplicate_rows = crime[crime.duplicated(keep=False)]
print(f"Total exact duplicates found: {duplicate_rows.shape[0]}")

Total exact duplicates found: 31898


In [ ]:
# Group duplicates and check for value differences across columns
duplicate_groups = duplicate_rows.groupby('offense_date')
for name, group in duplicate_groups:
    print(f"\nGroup: {name}")
    display(group)

In [149]:
# Drop exact duplicates and keep only the first occurrence
crime = crime.drop_duplicates()
duplicate_rows = crime[crime.duplicated(keep=False)]
print(f"Total exact duplicates found: {duplicate_rows.shape[0]}")

Total exact duplicates found: 0


In [150]:
# We now have a "complete" dataset

crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360042 entries, 0 to 524389
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ucr_desc             360042 non-null  object        
 1   time_block           360042 non-null  object        
 2   dow                  360042 non-null  int64         
 3   hour_of_day          360042 non-null  int64         
 4   offense_date         360042 non-null  datetime64[ns]
 5   census_tract         360042 non-null  object        
 6   census_block         360042 non-null  object        
 7   offense_year         360042 non-null  int64         
 8   temp_max             360042 non-null  float64       
 9   temp_min             360042 non-null  float64       
 10  daylight_duration    360042 non-null  float64       
 11  precipitation_sum    360042 non-null  float64       
 12  precipitation_hours  360042 non-null  float64       
 13  extracted_month    

In [151]:
from sklearn.preprocessing import LabelEncoder

mappings = {}

cols = ['ucr_desc', 'offense_year', 'time_block', 'census_tract', 'census_block']
for col in cols:
    le = LabelEncoder()
    crime[col + '_numeric'] = le.fit_transform(crime[col])
    
    mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    
for col, mapping in mappings.items():
    print(f"{col}: {mapping}")
    print("\n")

ucr_desc: {'1C': 0, 'All Other Offenses': 1, 'Arson': 2, 'Assault': 3, 'Burglary': 4, 'Disorderly Conduct': 5, 'Driving Under The Influence': 6, 'Drug Abuse Violations': 7, 'Drunkenness': 8, 'Embezzlement': 9, 'Fel Assault': 10, 'Forgery & Counterfeiting': 11, 'Fraud': 12, 'GTMV': 13, 'Gambling': 14, 'Homicide': 15, 'Liquor Laws': 16, 'Offense Against Family/Children': 17, 'Property Damaged': 18, 'Prostitution': 19, 'Rape': 20, 'Robbery': 21, 'Sex Offenses': 22, 'Stolen Property': 23, 'Theft': 24, 'Traffic Violations': 25, 'Vandalism': 26, 'Weapons': 27}


offense_year: {2018: 0, 2019: 1, 2020: 2, 2021: 3, 2022: 4}


time_block: {'Day': 0, 'Early Night': 1, 'Morning': 2, 'Night': 3}


census_tract: {'Census Tract 1011.01': 0, 'Census Tract 1011.02': 1, 'Census Tract 1012.01': 2, 'Census Tract 1013': 3, 'Census Tract 1014': 4, 'Census Tract 1015.01': 5, 'Census Tract 1016.03': 6, 'Census Tract 1017': 7, 'Census Tract 1018': 8, 'Census Tract 1019.01': 9, 'Census Tract 1021.01': 10, 'Cens

In [152]:
# Drop last remaining unneeded columns
# Only remaining column that may be unnessary is OffenseDate
# It is useful to have a column in datetime format

cols = ['ucr_desc', 'offense_year', 'time_block', 'census_tract', 'census_block']

crime = crime.drop(columns=cols)

In [153]:
duplicate_rows = crime[crime.duplicated(keep=False)]
print(f"Total exact duplicates found: {duplicate_rows.shape[0]}")

Total exact duplicates found: 0


In [154]:
# Important to note that we do not perform any feature scaling here
# This is because random forests are scale-invariant

crime.to_csv("crime_weather_preprocessed", index=False)